In [ ]:
!pip install yfinance tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path = "/content/drive/MyDrive/companyfacts.zip"

In [ ]:
import zipfile, json, os
import pandas as pd
from tqdm import tqdm

extract_dir = "/content/companyfacts"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Unzipped to:", extract_dir)

Unzipped to: /content/companyfacts


In [ ]:
import os

files = []
for root, _, f_list in os.walk(extract_dir):
    for f in f_list:
        if f.endswith(".json"):
            files.append(os.path.join(root, f))

print("Total JSON files:", len(files))


Total JSON files: 18898


In [ ]:
import os
import json
from collections import defaultdict
from tqdm import tqdm


# keyword rules for categories
CATEGORIES = {
    "net_income": ["netincome", "profitloss"],
    "revenue": ["revenue", "sales"],
    "assets": ["asset"],
    "liabilities": ["liabilit"],
    "equity": ["equity", "stockholders"],
    "shares_outstanding": ["sharesoutstanding", "commonstock"],
    "public_float": ["publicfloat"]
}

# store tag counts
tag_counts = defaultdict(int)

print(" Scanning JSON files...")
for root, _, files in os.walk(extract_dir):
    for file in tqdm(files):
        if file.endswith(".json"):
            with open(os.path.join(root, file)) as f:
                data = json.load(f)
                for ns, facts in data.get("facts", {}).items():
                    for tag in facts.keys():
                        tag_counts[tag.lower()] += 1  # lowercase for matching

print(f"\nFound {len(tag_counts)} unique tags")

# assign tags into categories
tag_map = defaultdict(list)

for tag, count in sorted(tag_counts.items(), key=lambda x: x[1], reverse=True):
    for cat, keywords in CATEGORIES.items():
        if any(kw in tag for kw in keywords):
            tag_map[cat].append(tag)
            break

# show results
for cat, tags in tag_map.items():
    print(f"\n {cat} ({len(tags)} tags):")

# save to JSON for later refinement
with open("TAG_MAP.json", "w") as f:
    json.dump(tag_map, f, indent=2)



🔍 Scanning JSON files...


100%|██████████| 18898/18898 [08:12<00:00, 38.40it/s]


Found 13552 unique tags

 assets (1804 tags):

 liabilities (1132 tags):

 net_income (143 tags):

 shares_outstanding (84 tags):

 equity (466 tags):

 public_float (1 tags):

 revenue (722 tags):


In [ ]:
import json
import os

# load the auto-built TAG_MAP
with open("TAG_MAP.json") as f:
    TAG_MAP = json.load(f)

def extract_metrics(data, cik, ticker, entity):
    records = []
    facts = data.get("facts", {})

    # loop through each target category (net_income, revenue, etc.)
    for category, tag_list in TAG_MAP.items():
        value = None
        year = None

        for ns, ns_facts in facts.items():
            # lowercase mapping of SEC tags
            ns_facts_lower = {k.lower(): v for k, v in ns_facts.items()}

            for tag in tag_list:
                if tag in ns_facts_lower:   # lowercase match
                    fact = ns_facts_lower[tag]

                    if "units" in fact:
                        for unit, entries in fact["units"].items():
                            # pick the latest entry with 'end'
                            for entry in entries:
                                if "end" in entry and entry.get("val") is not None:
                                    value = entry["val"]
                                    year = entry["end"][:4]
                                    break
                            if value is not None:
                                break
                if value is not None:
                    break
            if value is not None:
                break

        if value is not None:
            records.append({
                "cik": cik,
                "ticker": ticker if ticker else "",
                "entity": entity,
                "year": year,
                "metric": category,
                "value": value
            })

    return records


In [ ]:
sample_file = "/content/companyfacts/CIK0000001750.json"
with open(sample_file) as f:
    data = json.load(f)

cik = data.get("cik")
ticker = data.get("entityIdentifier", {}).get("ticker")
entity = data.get("entityName")

print(extract_metrics(data, cik, ticker, entity))


[{'cik': 1750, 'ticker': '', 'entity': 'AAR CORP', 'year': '2010', 'metric': 'assets', 'value': 1501042000}, {'cik': 1750, 'ticker': '', 'entity': 'AAR CORP', 'year': '2010', 'metric': 'liabilities', 'value': 1501042000}, {'cik': 1750, 'ticker': '', 'entity': 'AAR CORP', 'year': '2009', 'metric': 'net_income', 'value': 56772000}, {'cik': 1750, 'ticker': '', 'entity': 'AAR CORP', 'year': '2010', 'metric': 'shares_outstanding', 'value': 39662816}, {'cik': 1750, 'ticker': '', 'entity': 'AAR CORP', 'year': '2010', 'metric': 'equity', 'value': 746906000}, {'cik': 1750, 'ticker': '', 'entity': 'AAR CORP', 'year': '2010', 'metric': 'public_float', 'value': 934567976}, {'cik': 1750, 'ticker': '', 'entity': 'AAR CORP', 'year': '2020', 'metric': 'revenue', 'value': 9800000}]


In [ ]:
root = "/content/companyfacts"
all_records = []

for file in os.listdir(root):
    if file.endswith(".json"):
        with open(os.path.join(root, file)) as f:
            data = json.load(f)
            cik = data.get("cik")
            ticker = data.get("entityIdentifier", {}).get("ticker")
            entity = data.get("entityName")

            recs = extract_metrics(data, cik, ticker, entity)
            all_records.extend(recs)

import pandas as pd
df = pd.DataFrame(all_records)

print(df.head())


          cik ticker                  entity  year              metric  \
0  0002033615         Magnolia Bancorp, Inc.  2023              assets   
1  0002033615         Magnolia Bancorp, Inc.  2023         liabilities   
2  0002033615         Magnolia Bancorp, Inc.  2023          net_income   
3  0002033615         Magnolia Bancorp, Inc.  2024  shares_outstanding   
4  0002033615         Magnolia Bancorp, Inc.  2022              equity   

        value  
0  35803000.0  
1  35803000.0  
2     80000.0  
3         0.0  
4  13930000.0  


In [ ]:
print(df.shape)


(109955, 6)


In [ ]:
import pandas as pd

print("Before pivot:", df.shape)

# pivot to wide format (one row per cik/ticker/entity/year)
df_wide = df.pivot_table(
    index=["cik", "ticker", "entity", "year"],
    columns="metric",
    values="value",
    aggfunc="first"
).reset_index()

print("After pivot:", df_wide.shape)
print(df_wide.head())

# save to CSV
output_file = "company_metrics.csv"
df_wide.to_csv(output_file, index=False)
print(f" Saved to {output_file}")


Before pivot: (109955, 6)
After pivot: (52497, 11)
metric   cik ticker               entity  year        assets        equity  \
0       1750                    AAR CORP  2009           NaN           NaN   
1       1750                    AAR CORP  2010  1.501042e+09  7.469060e+08   
2       1750                    AAR CORP  2020           NaN           NaN   
3       1800         ABBOTT LABORATORIES  2007  3.971392e+10  1.777854e+10   
4       1800         ABBOTT LABORATORIES  2008           NaN           NaN   

metric   liabilities  net_income  public_float    revenue  shares_outstanding  
0                NaN  56772000.0           NaN        NaN                 NaN  
1       1.501042e+09         NaN  9.345680e+08        NaN          39662816.0  
2                NaN         NaN           NaN  9800000.0                 NaN  
3       3.971392e+10         NaN           NaN        NaN                 NaN  
4                NaN         NaN  7.877102e+10        NaN                 NaN  


In [ ]:
import pandas as pd

df = pd.read_csv("company_metrics.csv")

# --- Add financial ratios ---
df["ROE"] = df["net_income"] / df["equity"]          # Return on Equity
df["ROA"] = df["net_income"] / df["assets"]          # Return on Assets
df["DebtToEquity"] = df["liabilities"] / df["equity"]
df["ProfitMargin"] = df["net_income"] / df["revenue"]
df["AssetTurnover"] = df["revenue"] / df["assets"]
df["EquityRatio"] = df["equity"] / df["assets"]

#Earnings per Share (needs shares outstanding)
df["EPS"] = df["net_income"] / df["shares_outstanding"]


# Save enriched dataset
df.to_csv("company_metrics_with_ratios.csv", index=False)

print("saved with ratios:", df.shape)
print(df.head())


saved with ratios: (52497, 18)
    cik  ticker               entity  year        assets        equity  \
0  1750     NaN             AAR CORP  2009           NaN           NaN   
1  1750     NaN             AAR CORP  2010  1.501042e+09  7.469060e+08   
2  1750     NaN             AAR CORP  2020           NaN           NaN   
3  1800     NaN  ABBOTT LABORATORIES  2007  3.971392e+10  1.777854e+10   
4  1800     NaN  ABBOTT LABORATORIES  2008           NaN           NaN   

    liabilities  net_income  public_float    revenue  shares_outstanding  ROE  \
0           NaN  56772000.0           NaN        NaN                 NaN  NaN   
1  1.501042e+09         NaN  9.345680e+08        NaN          39662816.0  NaN   
2           NaN         NaN           NaN  9800000.0                 NaN  NaN   
3  3.971392e+10         NaN           NaN        NaN                 NaN  NaN   
4           NaN         NaN  7.877102e+10        NaN                 NaN  NaN   

   ROA  DebtToEquity  ProfitMargin  A

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

# 1. Fill missing raw values
def fill_missing_smart(df):
    df_out = df.copy()
    for col in ["assets", "liabilities", "equity", "net_income", "public_float",
                "revenue", "shares_outstanding"]:
        if col in df_out.columns:
            # fill per company (cik or ticker)
            df_out[col] = (
                df_out.groupby("cik")[col]
                .transform(lambda x: x.ffill().bfill().fillna(x.median()))
            )
    return df_out

# 2. Recompute ratios
def add_ratios(df):
    df = df.copy()

    df["ROE"] = df["net_income"] / df["equity"].replace(0, np.nan)
    df["ROA"] = df["net_income"] / df["assets"].replace(0, np.nan)
    df["DebtToEquity"] = df["liabilities"] / df["equity"].replace(0, np.nan)
    df["CurrentRatio"] = df["assets"] / df["liabilities"].replace(0, np.nan)
    df["ProfitMargin"] = df["net_income"] / df["revenue"].replace(0, np.nan)
    df["AssetTurnover"] = df["revenue"] / df["assets"].replace(0, np.nan)
    df["EquityRatio"] = df["equity"] / df["assets"].replace(0, np.nan)
    df["EPS"] = df["net_income"] / df["shares_outstanding"].replace(0, np.nan)
    df["FloatPct"] = df["public_float"] / (df["shares_outstanding"] * df["equity"] / df["equity"])

    return df

# 3. Fallback with yfinance
def fill_ratios_from_yf(df):
    df = df.copy()

    for ticker in df["ticker"].dropna().unique():
        try:
            t = yf.Ticker(ticker)
            info = t.info
        except Exception:
            continue

        mask = df["ticker"] == ticker
        if df.loc[mask, "ROE"].isna().any() and "returnOnEquity" in info:
            df.loc[mask, "ROE"] = df.loc[mask, "ROE"].fillna(info["returnOnEquity"])
        if df.loc[mask, "ROA"].isna().any() and "returnOnAssets" in info:
            df.loc[mask, "ROA"] = df.loc[mask, "ROA"].fillna(info["returnOnAssets"])
        if df.loc[mask, "DebtToEquity"].isna().any() and "debtToEquity" in info:
            df.loc[mask, "DebtToEquity"] = df.loc[mask, "DebtToEquity"].fillna(info["debtToEquity"])
        if df.loc[mask, "ProfitMargin"].isna().any() and "profitMargins" in info:
            df.loc[mask, "ProfitMargin"] = df.loc[mask, "ProfitMargin"].fillna(info["profitMargins"])
        # EPS
        if df.loc[mask, "EPS"].isna().any() and "trailingEps" in info:
            df.loc[mask, "EPS"] = df.loc[mask, "EPS"].fillna(info["trailingEps"])

    return df

# Full pipeline
def process_financials(df_raw):
    df1 = fill_missing_smart(df_raw)
    df2 = add_ratios(df1)
    df3 = fill_ratios_from_yf(df2)
    return df3

df_processed = process_financials(df)
print(df_processed.head());

    cik  ticker               entity  year        assets        equity  \
0  1750     NaN             AAR CORP  2009  1.501042e+09  7.469060e+08   
1  1750     NaN             AAR CORP  2010  1.501042e+09  7.469060e+08   
2  1750     NaN             AAR CORP  2020  1.501042e+09  7.469060e+08   
3  1800     NaN  ABBOTT LABORATORIES  2007  3.971392e+10  1.777854e+10   
4  1800     NaN  ABBOTT LABORATORIES  2008  3.971392e+10  1.777854e+10   

    liabilities    net_income  public_float       revenue  shares_outstanding  \
0  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06        3.966282e+07   
1  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06        3.966282e+07   
2  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06        3.966282e+07   
3  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10        1.545912e+09   
4  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10        1.545912e+09   

        ROE       ROA  DebtToEquity  ProfitMargin  AssetTurnover  Eq

In [ ]:
df_processed.to_csv("financials_with_ratios.csv", index=False)

In [ ]:
import requests
import pandas as pd

url = "https://www.sec.gov/files/company_tickers.json"
headers = {"User-Agent": "Brave/5.0 (compatible; MyApp/1.0; simaran@gmail.com)"}

resp = requests.get(url, headers=headers)
resp.raise_for_status()

tickers = resp.json()
df_map = pd.DataFrame.from_dict(tickers, orient="index")
df_map = df_map.rename(columns={"cik_str":"cik", "ticker":"ticker", "title":"entity"})
df_map["cik"] = df_map["cik"].astype(int)

print(df_map.head())


       cik ticker          entity
0  1045810   NVDA     NVIDIA CORP
1   789019   MSFT  MICROSOFT CORP
2   320193   AAPL      Apple Inc.
3  1652044  GOOGL   Alphabet Inc.
4  1018724   AMZN  AMAZON COM INC


In [ ]:
# Merge ticker mapping
df_processed = df_processed.merge(df_map[["cik", "ticker"]], on="cik", how="left", suffixes=("", "_map"))

# Prefer SEC ticker if missing in extracted data
df_processed["ticker"] = df_processed["ticker"].fillna(df_processed["ticker_map"])
df_processed = df_processed.drop(columns=["ticker_map"])


In [ ]:
print(df_processed.head())

    cik ticker               entity  year        assets        equity  \
0  1750    AIR             AAR CORP  2009  1.501042e+09  7.469060e+08   
1  1750    AIR             AAR CORP  2010  1.501042e+09  7.469060e+08   
2  1750    AIR             AAR CORP  2020  1.501042e+09  7.469060e+08   
3  1800    ABT  ABBOTT LABORATORIES  2007  3.971392e+10  1.777854e+10   
4  1800    ABT  ABBOTT LABORATORIES  2008  3.971392e+10  1.777854e+10   

    liabilities    net_income  public_float       revenue  shares_outstanding  \
0  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06        3.966282e+07   
1  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06        3.966282e+07   
2  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06        3.966282e+07   
3  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10        1.545912e+09   
4  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10        1.545912e+09   

        ROE       ROA  DebtToEquity  ProfitMargin  AssetTurnover  EquityRa

In [ ]:
import yfinance as yf
import numpy as np

def add_extra_ratios(df):
    pe_list, pb_list, div_yield_list, roi_list = [], [], [], []

    for _, row in df.iterrows():
        ticker = row["ticker"]
        eps = row.get("EPS", np.nan)
        equity = row.get("equity", np.nan)
        shares = row.get("shares_outstanding", np.nan)
        net_income = row.get("net_income", np.nan)
        assets = row.get("assets", np.nan)

        if ticker and isinstance(ticker, str):
            try:
                stock = yf.Ticker(ticker)
                info = stock.info
                price = info.get("currentPrice", np.nan)

                # --- P/E Ratio ---
                pe = price / eps if eps and eps != 0 else np.nan

                # --- P/B Ratio ---
                book_value_per_share = (equity / shares) if shares and equity else np.nan
                pb = price / book_value_per_share if book_value_per_share else np.nan

                # --- Dividend Yield ---
                div_yield = info.get("dividendYield", np.nan)

            except Exception:
                pe, pb, div_yield = np.nan, np.nan, np.nan
        else:
            pe, pb, div_yield = np.nan, np.nan, np.nan

        # --- ROI (Net Income / Assets) ---
        roi = (net_income / assets) if assets and net_income else np.nan

        pe_list.append(pe)
        pb_list.append(pb)
        div_yield_list.append(div_yield)
        roi_list.append(roi)

    df["PE_Ratio"] = pe_list
    df["PB_Ratio"] = pb_list
    df["Dividend_Yield"] = div_yield_list
    df["ROI"] = roi_list
    return df


In [ ]:
add_extra_ratios(df_processed)

,cik,ticker,entity,year,assets,equity,liabilities,net_income,public_float,revenue,...,ProfitMargin,AssetTurnover,EquityRatio,EPS,CurrentRatio,FloatPct,PE_Ratio,PB_Ratio,Dividend_Yield,ROI
0,1750,AIR,AAR CORP,2009,1.501042e+09,7.469060e+08,1.501042e+09,5.677200e+07,9.345680e+08,9.800000e+06,...,5.793061,0.006529,0.497592,1.431366,1.0,23.562825,52.872577,4.018822,NaN,0.037822
1,1750,AIR,AAR CORP,2010,1.501042e+09,7.469060e+08,1.501042e+09,5.677200e+07,9.345680e+08,9.800000e+06,...,5.793061,0.006529,0.497592,1.431366,1.0,23.562825,52.872577,4.018822,NaN,0.037822
2,1750,AIR,AAR CORP,2020,1.501042e+09,7.469060e+08,1.501042e+09,5.677200e+07,9.345680e+08,9.800000e+06,...,5.793061,0.006529,0.497592,1.431366,1.0,23.562825,52.872577,4.018822,NaN,0.037822
3,1800,ABT,ABBOTT LABORATORIES,2007,3.971392e+10,1.777854e+10,3.971392e+10,4.728000e+09,7.877102e+10,2.085300e+10,...,0.226730,0.525080,0.447665,3.058388,1.0,50.954384,43.218847,11.493559,1.79,0.119051
4,1800,ABT,ABBOTT LABORATORIES,2008,3.971392e+10,1.777854e+10,3.971392e+10,4.728000e+09,7.877102e+10,2.085300e+10,...,0.226730,0.525080,0.447665,3.058388,1.0,50.954384,43.218847,11.493559,1.79,0.119051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58434,2065948,SOCAU,SOLARIUS CAPITAL ACQUISITION CORP.,2025,1.562090e+05,0.000000e+00,1.562090e+05,-7.701400e+04,NaN,NaN,...,NaN,NaN,0.000000,-0.015403,1.0,NaN,NaN,NaN,NaN,-0.493019
58435,2066067,NaN,Orielle Acquisition Corp.,2025,1.029900e+04,-1.421600e+04,1.029900e+04,-1.471600e+04,NaN,NaN,...,NaN,NaN,-1.380328,-0.002943,1.0,NaN,NaN,NaN,NaN,-1.428877
58436,2069238,PAII,PYROPHYTE ACQUISITION CORP. II,2025,2.171040e+05,-7.700000e+04,2.171040e+05,-1.020000e+05,NaN,NaN,...,NaN,NaN,-0.354669,-0.016166,1.0,NaN,NaN,NaN,NaN,-0.469821
58437,2069238,PAII-UN,PYROPHYTE ACQUISITION CORP. II,2025,2.171040e+05,-7.700000e+04,2.171040e+05,-1.020000e+05,NaN,NaN,...,NaN,NaN,-0.354669,-0.016166,1.0,NaN,NaN,NaN,NaN,-0.469821


In [ ]:
print(df_processed.head())

    cik ticker               entity  year        assets        equity  \
0  1750    AIR             AAR CORP  2009  1.501042e+09  7.469060e+08   
1  1750    AIR             AAR CORP  2010  1.501042e+09  7.469060e+08   
2  1750    AIR             AAR CORP  2020  1.501042e+09  7.469060e+08   
3  1800    ABT  ABBOTT LABORATORIES  2007  3.971392e+10  1.777854e+10   
4  1800    ABT  ABBOTT LABORATORIES  2008  3.971392e+10  1.777854e+10   

    liabilities    net_income  public_float       revenue  ...  ProfitMargin  \
0  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...      5.793061   
1  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...      5.793061   
2  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...      5.793061   
3  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10  ...      0.226730   
4  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10  ...      0.226730   

   AssetTurnover  EquityRatio       EPS  CurrentRatio   FloatPct   PE_Ratio  \
0

In [ ]:
df_processed["Dividend_Yield"] = df_processed["Dividend_Yield"].fillna(0)

In [ ]:
print(df_processed.shape)

(58439, 24)


In [ ]:
# Drop rows where ticker is missing
# df_processed = df_processed[df_processed["ticker"].notna()]


In [ ]:
print(df_processed.shape)

(58439, 24)


In [ ]:
df_processed = df_processed.rename(columns={
    "PE_Ratio": "P/E Ratio",
})

In [ ]:
print(df_processed.head())

    cik ticker               entity  year        assets        equity  \
0  1750    AIR             AAR CORP  2009  1.501042e+09  7.469060e+08   
1  1750    AIR             AAR CORP  2010  1.501042e+09  7.469060e+08   
2  1750    AIR             AAR CORP  2020  1.501042e+09  7.469060e+08   
3  1800    ABT  ABBOTT LABORATORIES  2007  3.971392e+10  1.777854e+10   
4  1800    ABT  ABBOTT LABORATORIES  2008  3.971392e+10  1.777854e+10   

    liabilities    net_income  public_float       revenue  ...  ProfitMargin  \
0  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...      5.793061   
1  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...      5.793061   
2  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...      5.793061   
3  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10  ...      0.226730   
4  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10  ...      0.226730   

   AssetTurnover  EquityRatio       EPS  CurrentRatio   FloatPct  P/E Ratio  \
0

In [ ]:
import pandas as pd

print(df_processed.info())       # Check for nulls and data types
print(df_processed.isnull().sum())  # Count of nulls in each column
print(df_processed.duplicated().sum())  # Check for duplicates


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58439 entries, 0 to 58438
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cik                 58439 non-null  int64  
 1   ticker              28352 non-null  object 
 2   entity              58439 non-null  object 
 3   year                58439 non-null  int64  
 4   assets              58330 non-null  float64
 5   equity              57979 non-null  float64
 6   liabilities         58386 non-null  float64
 7   net_income          58293 non-null  float64
 8   public_float        49420 non-null  float64
 9   revenue             54457 non-null  float64
 10  shares_outstanding  58052 non-null  float64
 11  ROE                 54040 non-null  float64
 12  ROA                 57119 non-null  float64
 13  DebtToEquity        54097 non-null  float64
 14  ProfitMargin        48193 non-null  float64
 15  AssetTurnover       53500 non-null  float64
 16  Equi

In [ ]:
# Fill minor & moderate missing values with median
numerical_cols = ['assets', 'equity', 'liabilities', 'net_income', 'public_float',
                  'revenue', 'shares_outstanding', 'ROE', 'ROA', 'DebtToEquity',
                  'ProfitMargin', 'AssetTurnover', 'EquityRatio', 'EPS',
                  'CurrentRatio', 'FloatPct', 'ROI']

for col in numerical_cols:
    if col in df_processed.columns:
        median_val = df_processed[col].median()
        df_processed[col].fillna(median_val, inplace=True)

# Remove duplicates
df_processed.drop_duplicates(inplace=True)


/tmp/ipython-input-215498159.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].fillna(median_val, inplace=True)


In [ ]:
print(df_processed.shape)

(58439, 24)


In [ ]:
import yfinance as yf
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Ensure tickers are strings and remove NaNs
df_processed = df_processed[df_processed['ticker'].notnull()].copy()
df_processed['ticker'] = df_processed['ticker'].astype(str)

tickers = df_processed['ticker'].unique()
years = df_processed['year'].unique()

# Function to fetch yearly average prices for a single ticker
def fetch_ticker_prices(ticker):
    if ticker.strip() == '':
        return None
    try:
        data = yf.Ticker(ticker)
        hist = data.history(start="1900-01-01", end="2030-12-31")
        if hist.empty:
            return None
        hist['year'] = hist.index.year
        yearly_avg = hist.groupby('year')['Close'].mean().reset_index()
        yearly_avg['ticker'] = ticker
        return yearly_avg
    except:
        return None

# Use ThreadPoolExecutor to fetch tickers in parallel
price_records = []
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_ticker = {executor.submit(fetch_ticker_prices, ticker): ticker for ticker in tickers}
    for future in as_completed(future_to_ticker):
        result = future.result()
        if result is not None:
            price_records.append(result)

# Combine results into a DataFrame
df_prices = pd.concat(price_records, ignore_index=True)
df_prices.rename(columns={'Close': 'ticker_price'}, inplace=True)

# Keep only years present in dataset
df_prices = df_prices[df_prices['year'].isin(years)]

# Fill missing years per ticker using nearest available price
df_prices = df_prices.sort_values(['ticker', 'year'])
df_prices['ticker_price'] = df_prices.groupby('ticker')['ticker_price'].ffill().bfill()

# Merge with main dataset
df_processed = df_processed.merge(df_prices[['ticker','year','ticker_price']], on=['ticker','year'], how='left')

# Save for caching
df_prices.to_csv("ticker_prices.csv", index=False)

print(df_processed[['ticker','year','ticker_price']].head(10))


  ticker  year  ticker_price
0    AIR  2009           NaN
1    AIR  2010           NaN
2    AIR  2020           NaN
3    ABT  2007           NaN
4    ABT  2008           NaN
5    ABT  2009           NaN
6    ABT  2011           NaN
7    ABT  2016           NaN
8    ACU  2009           NaN
9    ACU  2010           NaN


In [ ]:
# Replace '.' with '-' for tickers like BRK.B → BRK-B
df_processed['ticker'] = df_processed['ticker'].str.replace('.', '-', regex=False)


In [ ]:
print(df_prices.shape)
print(df_prices.head())


(516, 3)
     year  ticker_price ticker
187  2017      9.755820   ACEL
188  2018      9.935649   ACEL
189  2019     10.402464   ACEL
190  2020     10.331225   ACEL
191  2021     11.831508   ACEL


In [ ]:
# First, sort
df_prices = df_prices.sort_values(['ticker', 'year'])

# Forward and backward fill per ticker
df_prices['ticker_price'] = df_prices.groupby('ticker')['ticker_price'].ffill().bfill()

# Merge with main dataset
df_processed = df_processed.merge(df_prices[['ticker','year','ticker_price']], on=['ticker','year'], how='left')

# Fill remaining NaNs with overall median
median_price = df_processed['ticker_price'].median()
df_processed['ticker_price'].fillna(median_price, inplace=True)


/tmp/ipython-input-2144405430.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed['ticker_price'].fillna(median_price, inplace=True)


In [ ]:
import numpy as np

df_processed['P/E Ratio'] = df_processed.apply(
    lambda x: x['P/E Ratio'] if pd.notnull(x['P/E Ratio'])
    else (x['ticker_price'] / x['EPS'] if x['EPS'] and x['EPS'] > 0 else np.nan), axis=1
)

df_processed['PB_Ratio'] = df_processed.apply(
    lambda x: x['PB_Ratio'] if pd.notnull(x['PB_Ratio'])
    else (x['ticker_price'] / (x['equity'] / x['shares_outstanding'])
          if x['shares_outstanding'] and x['shares_outstanding'] > 0 and x['equity'] and x['equity'] > 0
          else np.nan), axis=1
)


In [ ]:
print(df_processed.shape)
print(df_processed.head())

(28352, 27)
    cik ticker               entity  year        assets        equity  \
0  1750    AIR             AAR CORP  2009  1.501042e+09  7.469060e+08   
1  1750    AIR             AAR CORP  2010  1.501042e+09  7.469060e+08   
2  1750    AIR             AAR CORP  2020  1.501042e+09  7.469060e+08   
3  1800    ABT  ABBOTT LABORATORIES  2007  3.971392e+10  1.777854e+10   
4  1800    ABT  ABBOTT LABORATORIES  2008  3.971392e+10  1.777854e+10   

    liabilities    net_income  public_float       revenue  ...       EPS  \
0  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...  1.431366   
1  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...  1.431366   
2  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...  1.431366   
3  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10  ...  3.058388   
4  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10  ...  3.058388   

   CurrentRatio   FloatPct  P/E Ratio   PB_Ratio  Dividend_Yield       ROI  \
0           1.

In [ ]:
import numpy as np

# Fill missing P/E Ratio
df_processed['P/E Ratio'] = np.where(
    pd.notnull(df_processed['P/E Ratio']),  # keep existing if present
    df_processed['P/E Ratio'],
    np.where(df_processed['EPS'] > 0, df_processed['ticker_price'] / df_processed['EPS'], np.nan)
)

# Fill missing P/B Ratio
df_processed['PB_Ratio'] = np.where(
    pd.notnull(df_processed['PB_Ratio']),  # keep existing if present
    df_processed['PB_Ratio'],
    np.where(
        (df_processed['equity'] > 0) & (df_processed['shares_outstanding'] > 0),
        df_processed['ticker_price'] / (df_processed['equity'] / df_processed['shares_outstanding']),
        np.nan
    )
)


In [ ]:
print(df_processed.shape)
print(df_processed.head())

(28352, 27)
    cik ticker               entity  year        assets        equity  \
0  1750    AIR             AAR CORP  2009  1.501042e+09  7.469060e+08   
1  1750    AIR             AAR CORP  2010  1.501042e+09  7.469060e+08   
2  1750    AIR             AAR CORP  2020  1.501042e+09  7.469060e+08   
3  1800    ABT  ABBOTT LABORATORIES  2007  3.971392e+10  1.777854e+10   
4  1800    ABT  ABBOTT LABORATORIES  2008  3.971392e+10  1.777854e+10   

    liabilities    net_income  public_float       revenue  ...       EPS  \
0  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...  1.431366   
1  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...  1.431366   
2  1.501042e+09  5.677200e+07  9.345680e+08  9.800000e+06  ...  1.431366   
3  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10  ...  3.058388   
4  3.971392e+10  4.728000e+09  7.877102e+10  2.085300e+10  ...  3.058388   

   CurrentRatio   FloatPct  P/E Ratio   PB_Ratio  Dividend_Yield       ROI  \
0           1.

In [ ]:
import numpy as np

# Fill missing P/E Ratio
df_processed['P/E Ratio'] = np.where(
    pd.notnull(df_processed['P/E Ratio']),
    df_processed['P/E Ratio'],  # keep original if exists
    np.where(df_processed['EPS'] > 0, df_processed['ticker_price'] / df_processed['EPS'], np.nan)
)

# Fill missing P/B Ratio
df_processed['PB_Ratio'] = np.where(
    pd.notnull(df_processed['PB_Ratio']),
    df_processed['PB_Ratio'],  # keep original if exists
    np.where(
        (df_processed['equity'] > 0) & (df_processed['shares_outstanding'] > 0),
        df_processed['ticker_price'] / (df_processed['equity'] / df_processed['shares_outstanding']),
        np.nan
    )
)


In [ ]:
df_processed['P/E Ratio'].fillna(df_processed['P/E Ratio'].median(), inplace=True)
df_processed['PB_Ratio'].fillna(df_processed['PB_Ratio'].median(), inplace=True)


/tmp/ipython-input-882418333.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed['P/E Ratio'].fillna(df_processed['P/E Ratio'].median(), inplace=True)
/tmp/ipython-input-882418333.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].meth

In [ ]:
import pandas as pd

print(df_processed.info())       # Check for nulls and data types
print(df_processed.isnull().sum())  # Count of nulls in each column
print(df_processed.duplicated().sum())  # Check for duplicates


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28352 entries, 0 to 28351
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cik                 28352 non-null  int64  
 1   ticker              28352 non-null  object 
 2   entity              28352 non-null  object 
 3   year                28352 non-null  int64  
 4   assets              28352 non-null  float64
 5   equity              28352 non-null  float64
 6   liabilities         28352 non-null  float64
 7   net_income          28352 non-null  float64
 8   public_float        28352 non-null  float64
 9   revenue             28352 non-null  float64
 10  shares_outstanding  28352 non-null  float64
 11  ROE                 28352 non-null  float64
 12  ROA                 28352 non-null  float64
 13  DebtToEquity        28352 non-null  float64
 14  ProfitMargin        28352 non-null  float64
 15  AssetTurnover       28352 non-null  float64
 16  Equi

In [ ]:
# Save to CSV
df_processed.to_csv("fda-4_dataset.csv", index=False)
print(" CSV saved successfully")


✅ CSV saved successfully
